In [1]:
import os

import numpy as np
import pandas as pd
from scipy.stats import median_absolute_deviation
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras 
import tensorflow_addons as tfa
random_state = 1234
from keras.layers import Dropout
from numpy.random import seed
seed(random_state)
tf.random.set_seed(random_state)
from keras.models import load_model
import copy

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import optuna

In [3]:
from sklearn.preprocessing import StandardScaler
import random

In [4]:
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt

In [5]:
data_dir = r"D:\NIH\Mutational-Spectrum-master\Data\New\MSK_Impact_train"

In [6]:
def list_files_in_dir(dirname):
    dir_files = list()
    for root, _, files in os.walk(dirname):
        for file in files:
            dir_files.append(os.path.join(root, file))
    
    return dir_files

In [9]:
# With overfitting
def build_mlp_model(input_shape=(96,), n_hidden_layers=1, n_hidden_nodes=8, activation='relu',learning_rate=0.001, weight_decay = 0, dropout=0.2):
    # optimizer parameters
    loss = "binary_crossentropy"
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay = weight_decay)
    metrics = keras.metrics.AUC(name='auc')
    
    model = load_model(r"D:\NIH\Mutational-Spectrum-master\Scripts\Autoencoder\MSKIMPACT_encoder.h5")
    for layer in model.layers:
          layer.trainable = False
    
    for i in range(n_hidden_layers):
        model.add(keras.layers.Dense(n_hidden_nodes, activation=activation, name = 'dense_layer_' + str(i)))
    model.add(keras.layers.Dense(1, activation="sigmoid", name="prediction_layer"))
    
    # optimizer
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return model

In [8]:
# Without overfitting
def build_mlp_model( input_shape=(96,), learning_rate=0.001, weight_decay = 0, dropout=0.2):
    # optimizer parameters
    loss = "binary_crossentropy"
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay = weight_decay)
    metrics = keras.metrics.AUC(name='auc')
    
    model = load_model(r"D:\NIH\Mutational-Spectrum-master\Scripts\Autoencoder\MSKIMPACT_encoder.h5")
    for layer in model.layers:
          layer.trainable = False
    
#     for i in range(n_hidden_layers):
#         model.add(keras.layers.Dense(n_hidden_nodes, activation=activation, name = 'dense_layer_' + str(i)))
#     model.add(Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid", name="prediction_layer"))
    
    # optimizer
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return model

In [10]:
def get_row_indices_with_sum_zero(X):
    return X.index[(X.sum(axis=1) == 0)].tolist()

In [11]:
# Z-score normalization
def scale_row(X):
    for column in X.columns:
        X[column] = (X[column] -
                               X[column].mean()) / X[column].std() 
    return X

In [12]:
def dataset_generator(data_dir, num_files, y_col):

    # Would take eons to run if I didn't limit this to only the first file  
    for datafile in random.sample(list_files_in_dir(data_dir), num_files):
        data = pd.read_csv(datafile)
        X = data.iloc[:, :96]
        y = (data[y_col]).astype(np.int_)
#         X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
        X = scale_row(X)
    yield X, y

In [13]:
def scale_data(df):
    # Define the scaler 
    scaler = StandardScaler().fit(df)
    # Scales each individual row   
    df[df.columns] = scaler.fit_transform(df[df.columns])
    
    return df
    

In [14]:
def train_val_test_generator(data_dir, num_samples, num_folds, y_col, test_frac=0.1):
    val_frac = test_frac/(1.0 - test_frac)
    
    for X, y in dataset_generator(data_dir, num_samples, y_col):
        fold_data = list()
        skf = StratifiedKFold(n_splits=num_folds, shuffle = True, random_state = random_state)
#         skf = StratifiedKFold(n_splits=num_folds)
        
        for train_val_index, test_index in skf.split(X, y):
            X_test, y_test = X.iloc[test_index, :], y.iloc[test_index]
            X_train, X_val, y_train, y_val = train_test_split(
                X.iloc[train_val_index,:], y.iloc[train_val_index], 
                test_size=val_frac, 
                random_state=random_state, 
                stratify=y.iloc[train_val_index]
            )
            #X_train, X_val, X_test = scale_columns(X_train, X_val, X_test)
            fold_data.append(((X_train, y_train), 
                              (X_val, y_val), 
                              (X_test, y_test)))
        yield fold_data

In [15]:
def compute_mlp_performance(trial, data_dir=data_dir, n_folds=10, input_shape=(96,), 
                            n_hidden_layers=2, n_hidden_nodes=16, 
                            activation="relu", learning_rate=0.001, num_samples = 1, y_col = 'is_sig3_20', weight_decay = 0, dropout=0.2):
    aucs = list()
    models = list()
    
    # 60-20-20 split
    test_frac=1.0/float(n_folds)
#     fpr_list = []
#     tpr_list = []
#     roc_auc_list = []
    
    for folds_data in train_val_test_generator(data_dir, num_samples=num_samples, num_folds=n_folds, y_col=y_col):
        fold_aucs = list()
        fold_models = list()
        
        for fold_data in folds_data:
            # get data
            (X_train, y_train), (X_val, y_val), (X_test, y_test) = fold_data
            # build model and ensure that parameters passed in are within the normal range
            # if we don't type cast as integers, bayesian optimizer will guess float values
            model = build_mlp_model(input_shape, 
                                    trial.suggest_int('n_hidden_layers', 0, 8), 
                                    trial.suggest_int('n_hidden_nodes', 1, 128), 
                                    trial.suggest_categorical("activation", ["relu", "sigmoid", "softmax"]), 
                                    trial.suggest_float('learning_rate', 1e-6, 1e-1), 
                                    trial.suggest_float('weight_decay', 0, 1e-1))
#             model = build_mlp_model(input_shape, 
#                         trial.suggest_float('learning_rate', 1e-6, 1e-1), 
#                         trial.suggest_float('weight_decay', 0, 1e-1),
#                         trial.suggest_float('dropout', 0, 5e-1))

            model.fit(X_train, y_train, 
                      validation_data=(X_val, y_val), 
                      epochs=1000, batch_size=32, verbose=0,
                      callbacks=[keras.callbacks.EarlyStopping(monitor='val_auc', patience=5)])
            
            # evaluate
            y_score = model.evaluate(X_test, y_test, verbose=0)[1]
            fold_aucs.append(y_score)
            fold_models.append(model)
        aucs.append(fold_aucs)
        models.append(fold_models)
        
    # Gets median index value for all the different samples (rows)  
    medianIndices = [indices[len(aucs[0])//2] for indices in np.argsort(aucs, axis=1)]
    medianValues = [values[index] for values, index in zip(aucs, medianIndices)]
    
    # Gets the file which contains the median of median value
    fileInd = np.argsort(medianValues)[len(medianValues)//2]
    
    aucs = np.array(aucs)
    
    median_of_median_model = models[fileInd][medianIndices[fileInd]]
    median_of_median_auc = np.median(np.median(aucs, axis=1))
    mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
#     return median_of_median_auc, mad_of_mad_auc
    return median_of_median_auc
    return np.average(aucs)



In [15]:
def compute_overfitting_performance(trial, data_dir=data_dir, input_shape=(96,), 
                            n_hidden_layers=2, n_hidden_nodes=16, 
                            activation="relu", learning_rate=0.001, num_samples = 1, y_col = 'is_sig3_20', weight_decay = 0, dropout=0.2):
    aucs = None
    models = None
    
    # 60-20-20 split
    
    for X, y in dataset_generator(data_dir, num_samples, y_col):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
        # build model and ensure that parameters passed in are within the normal range
        # if we don't type cast as integers, bayesian optimizer will guess float values
        model = build_mlp_model(input_shape, 
                                trial.suggest_int('n_hidden_layers', 0, 8), 
                                trial.suggest_int('n_hidden_nodes', 1, 128), 
                                trial.suggest_categorical("activation", ["relu", "sigmoid", "softmax"]), 
                                trial.suggest_float('learning_rate', 1e-6, 1e-1), 
                                trial.suggest_float('weight_decay', 0, 1e-1))
#             model = build_mlp_model(input_shape, 
#                         trial.suggest_float('learning_rate', 1e-6, 1e-1), 
#                         trial.suggest_float('weight_decay', 0, 1e-1),
#                         trial.suggest_float('dropout', 0, 5e-1))

        model.fit(X_train, y_train, 
                  validation_data=(X_val, y_val), 
                  epochs=1000, batch_size=32, verbose=0,
                  callbacks=[keras.callbacks.EarlyStopping(monitor='val_auc', patience=5)])

        # evaluate
        aucs = model.evaluate(X_test, y_test, verbose=0)[1]
        
        models = model

        
    return aucs
    return np.average(aucs)



In [21]:
study_20 = optuna.create_study(direction='maximize')

[I 2022-06-24 14:04:27,683] A new study created in memory with name: no-name-933954ca-d890-4e79-acb4-fb25b35e1993


In [23]:
study_20.optimize(compute_mlp_performance, n_trials=100)
study_20.best_trial

C:\Users\aaron\AppData\Local\Temp\ipykernel_3508\1547987928.py:56: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
[I 2022-06-24 14:05:03,778] Trial 0 finished with value: 0.557580292224884 and parameters: {'n_hidden_layers': 1, 'n_hidden_nodes': 28, 'activation': 'sigmoid', 'learning_rate': 0.053395658434871276, 'weight_decay': 0.0074501225498356805}. Best is trial 0 with value: 0.557580292224884.


C:\Users\aaron\AppData\Local\Temp\ipykernel_3508\1547987928.py:56: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
[I 2022-06-24 14:05:29,347] Trial 1 finished with value: 0.5 and parameters: {'n_hidden_layers': 6, 'n_hidden_nodes': 31, 'activation': 'softmax', 'learning_rate': 0.054735720147404666, 'weight_decay': 0.08485234772320233}. Best is trial 0 with value: 0.557580292224884.


C:\Users\aaron\AppData\Local\Temp\ipykernel_3508\1547987928.py:56: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
[I 2022-06-24 14:05:52,504] Trial 2 finished with value: 0.4467852860689163 and parameters: {'n_hidden_layers': 1, 'n_hidden_nodes': 60, 'activation': 'softmax', 'learning_rate': 0.002996511837600134, 'weight_decay': 0.03508479126718724}. Best is trial 0 with value: 0.557580292224884.


C:\Users\aaron\AppData\Local\Temp\ipykernel_3508\1547987928.py:56: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
[I 2022-06-24 14:06:18,151] Trial 3 finished with value: 0.5 and parameters: {'n_hidden_layers': 7, 'n_hidden_nodes': 9, 'activation': 'relu', 'learning_rate': 0.04577476424473873, 'weight_decay': 0.05148150105102478}. Best is trial 0 with value: 0.557580292224884.


C:\Users\aaron\AppData\Local\Temp\ipykernel_3508\1547987928.py:56: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad_of_mad_auc = median_absolute_deviation(aucs, axis=1)
[I 2022-06-24 14:06:47,746] Trial 4 finished with value: 0.5 and parameters: {'n_hidden_layers': 4, 'n_hidden_nodes': 19, 'activation': 'relu', 'learning_rate': 0.0452209132443966, 'weight_decay': 0.06642691943672645}. Best is trial 0 with value: 0.557580292224884.


KeyboardInterrupt: 